# Capstone Project Week 3

#### Change this cell to **code** if run it for the first time
!pip install beautifulsoup4

!pip install lxml

!pip install html5lib

!pip install requests

In [1]:
from bs4 import BeautifulSoup
import requests

Extract the source of the wikipedia page and find the postal code table on that page:

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
table_head = table.thead
table_body = table.tbody
pcodes = table_body.find_all() 
print(table.tbody.tr.text)


Postal Code

Borough

Neighborhood



Wrtie the postal code table information into a csv file.

In [9]:
import csv
csv_file = open('table_file.csv' , 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postal Code' , 'Borough' , 'Neighborhood'])

pcodes = table_body.find_all('tr')
for item in pcodes:
    subitem_list = []
    for subitem in item.find_all('td'):
        subitem_list.append( subitem.text.split( "\n")[0] )
    csv_writer.writerow(subitem_list)
csv_file.close()

Create a data frame from the saved CSV file:

In [20]:
import pandas as pd
CanPostCode = pd.read_csv('table_file.csv')
print(CanPostCode.head())

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned                        NaN
1         M2A      Not assigned                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


Remove the rows whose Borough or Neighborhood values are **NAN**:

In [21]:
CanPostCode.drop( CanPostCode[ CanPostCode['Borough'] == 'Not assigned'].index , inplace = True)
CanPostCode.drop( CanPostCode[ CanPostCode['Neighborhood'] == 'Not assigned'].index , inplace = True)
CanPostCode.head(5)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
CanPostCode.shape

(103, 3)

In [23]:
CanPostCode_Stp2 = pd.read_csv('https://cocl.us/Geospatial_data')
CanPostCode_Stp2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
CanPostCode = pd.merge(CanPostCode, CanPostCode_Stp2, on="Postal Code" , how = 'left')
CanPostCode.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [25]:
print('The data fram has {} unique boroughs and {} neighborhood'.format(len(CanPostCode["Borough"].unique()),CanPostCode.shape[0]))
print(CanPostCode["Borough"].unique())
CanPostCode.groupby('Borough').count()

The data fram has 10 unique boroughs and 103 neighborhood
['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


In [27]:
import folium # map rendering library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# create map of Toronto using latitude and longitude values
n = 1
latitude = 43.6534817
longitude = -79.3839347
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(CanPostCode['Latitude'], CanPostCode['Longitude'], CanPostCode['Borough'], CanPostCode['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Find ten top venues in Toronto
Set up Forsquare:

In [28]:
CLIENT_ID = 'YUNBULII4XKK5MFG0AQLAL1MSRVLUOOMVC3NJRD2TGZE1RIQ' # your Foursquare ID
CLIENT_SECRET = 'RBJIE5V0BE4DZZKHELF0DMMG1CQSCLD1YR4NA5VO1RSSL4SR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: YUNBULII4XKK5MFG0AQLAL1MSRVLUOOMVC3NJRD2TGZE1RIQ
CLIENT_SECRET:RBJIE5V0BE4DZZKHELF0DMMG1CQSCLD1YR4NA5VO1RSSL4SR


In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Find top hundred venues in each neighborhood:

In [31]:
Toronto_venues = getNearbyVenues(names=CanPostCode['Neighborhood'],
                                   latitudes=CanPostCode['Latitude'],
                                   longitudes=CanPostCode['Longitude']
                                  )

In [39]:
top_venues = Toronto_venues.groupby('Venue Category').count().sort_values("Neighborhood" , ascending=False)
pd.set_option("display.max_rows", None, "display.max_columns", None)
top_venues['Count'] = top_venues['Venue']
top_venues[['Count']].head(10)

,Count
Venue Category,
Coffee Shop,174
Café,100
Restaurant,71
Park,54
Pizza Place,45
Italian Restaurant,45
Japanese Restaurant,42
Sandwich Place,41
Clothing Store,38


In [40]:
print('test')

test
